In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.preprocessing import OneHotEncoder
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import SVC

from sklearn.metrics import accuracy_score, log_loss

In [2]:
## Read csvs
train_df = pd.read_csv('train_ml.csv', index_col=0)
test_df = pd.read_csv('test_ml.csv', index_col=0)

In [3]:
train_df.head()

,date,org,tld,ccs,bcced,mail_type,images,urls,salutations,designation,chars_in_subject,chars_in_body,updates,personal,promotions,forums,purchases,travel,spam,social
0,"Mon, 15 Oct 2018 08:03:09 +0000 (UTC)",researchgatemail,net,0,0,multipart/alternative,4,28,0,1,47.0,25556,0,1,0,0,0,0,0,1
1,"Thu, 17 Apr 2014 09:12:33 -0700 (PDT)",no-ip,com,0,0,multipart/alternative,6,32,0,0,46.0,19930,1,1,0,0,0,0,0,0
2,"Thu, 27 Oct 2016 01:36:28 +0000",mail,goodreads.com,0,0,multipart/mixed,0,0,0,0,21.0,4,0,1,0,0,0,0,0,1
3,"Fri, 16 Oct 2020 08:06:31 +0000 (GMT)",insideapple,apple.com,0,0,multipart/alternative,108,171,0,0,52.0,96568,0,0,1,0,0,0,0,0
4,"Thu, 02 Jul 2015 10:16:46 +0000",twitter,com,0,0,multipart/alternative,20,166,0,0,81.0,95131,0,1,0,0,0,0,0,1


In [4]:
test_df.head()

,date,org,tld,ccs,bcced,mail_type,images,urls,salutations,designation,chars_in_subject,chars_in_body
0,"Thu, 13 Jul 2017 08:55:57 +0000",twitter,com,0,0,multipart/alternative,7,56,0,0,67.0,36243
1,"Sun, 30 Sep 2018 14:42:12 +0000",mailer,netflix.com,0,0,multipart/alternative,5,33,0,0,27.0,27015
2,"Mon, 13 Feb 2017 10:47:00 +0530",iiitd,ac.in,0,0,text/plain,0,2,1,0,22.0,788
3,"Thu, 16 Jun 2016 09:56:23 +0000",twitter,com,0,0,multipart/alternative,8,53,0,0,79.0,39504
4,"Mon, 18 Apr 2016 01:51:59 +0530",iiitd,ac.in,0,0,multipart/mixed,0,0,0,0,24.0,178773


In [5]:
train_df.describe()

,ccs,bcced,images,urls,salutations,designation,chars_in_subject,chars_in_body,updates,personal,promotions,forums,purchases,travel,spam,social
count,39671.000000,39671.000000,39671.000000,39671.000000,39671.000000,39671.000000,39660.000000,3.967100e+04,39671.000000,39671.000000,39671.000000,39671.000000,39671.000000,39671.000000,39671.000000,39671.000000
mean,0.378967,0.002974,9.806332,36.731088,0.395755,0.102317,51.442032,2.321781e+05,0.362406,0.809609,0.199768,0.155807,0.008293,0.002521,0.003832,0.100955
std,2.530353,0.054458,440.393286,144.379127,0.489018,0.303068,34.142227,2.251044e+06,0.480701,0.392614,0.399831,0.362676,0.090690,0.050144,0.061781,0.301273
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,4.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,4.500000,0.000000,0.000000,31.000000,4.903000e+03,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,2.000000,18.000000,0.000000,0.000000,43.000000,2.000500e+04,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,9.000000,47.000000,1.000000,0.000000,63.000000,4.800100e+04,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,155.000000,1.000000,83480.000000,21540.000000,1.000000,1.000000,528.000000,7.438108e+07,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [6]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 39671 entries, 0 to 39670
Data columns (total 20 columns):
date                39671 non-null object
org                 37994 non-null object
tld                 37992 non-null object
ccs                 39671 non-null int64
bcced               39671 non-null int64
mail_type           39573 non-null object
images              39671 non-null int64
urls                39671 non-null int64
salutations         39671 non-null int64
designation         39671 non-null int64
chars_in_subject    39660 non-null float64
chars_in_body       39671 non-null int64
updates             39671 non-null int64
personal            39671 non-null int64
promotions          39671 non-null int64
forums              39671 non-null int64
purchases           39671 non-null int64
travel              39671 non-null int64
spam                39671 non-null int64
social              39671 non-null int64
dtypes: float64(1), int64(15), object(4)
memory usage: 6.4+ MB


In [7]:
train_df.groupby(['tld','org'])[['updates','personal','promotions','forums','purchases','travel','spam','social']].mean()

,,updates,personal,promotions,forums,purchases,travel,spam,social
tld,org,,,,,,,,
21cf.com,digital,0.500000,1.000000,0.500000,0.000000,0.000000,0.00000,0.0,0.0
33mail.com,mailer1,1.000000,0.058824,0.000000,0.000000,0.000000,0.00000,0.0,0.0
4shared.com,mx2,1.000000,1.000000,0.000000,0.000000,0.000000,0.00000,0.0,0.0
Apple.com,InsideApple,0.545455,0.818182,0.454545,0.000000,0.000000,0.00000,0.0,0.0
BRASSRING.COM,TRM,1.000000,1.000000,0.000000,0.000000,0.000000,0.00000,0.0,0.0
GOV,STATE,1.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.0,0.0
ORG,ZOONIVERSE,0.866667,1.000000,0.066667,0.066667,0.000000,0.00000,0.0,0.0
aamaadmiparty.org,delhi,1.000000,1.000000,0.000000,0.000000,0.000000,0.00000,0.0,0.0
aau.dk,create,0.000000,1.000000,0.000000,0.000000,0.000000,0.00000,0.0,0.0


In [8]:
train_df.shape

(39671, 20)

In [9]:
train_df.dtypes.value_counts()

int64      15
object      4
float64     1
dtype: int64

In [10]:
train_df.dtypes

date                 object
org                  object
tld                  object
ccs                   int64
bcced                 int64
mail_type            object
images                int64
urls                  int64
salutations           int64
designation           int64
chars_in_subject    float64
chars_in_body         int64
updates               int64
personal              int64
promotions            int64
forums                int64
purchases             int64
travel                int64
spam                  int64
social                int64
dtype: object

In [11]:
def split_X_y(Xa):
    return Xa.drop(['updates','personal','promotions','forums','purchases','travel','spam','social'],axis=1), Xa[['updates','personal','promotions','forums','purchases','travel','spam','social']]

In [12]:
X,y=split_X_y(train_df)
X_test=test_df

In [13]:
columns=list(X.columns)
print(columns)

['date', 'org', 'tld', 'ccs', 'bcced', 'mail_type', 'images', 'urls', 'salutations', 'designation', 'chars_in_subject', 'chars_in_body']


In [14]:
categorical_features=X.select_dtypes(include=['object','category']).columns.tolist()
numerical_features=X.select_dtypes(exclude=['object','category']).columns.tolist()

In [15]:
print('columns:', columns)
print('-------------')
print('categorical features:', categorical_features)
print('-------------')
print('numericalfeatures:', numerical_features)

columns: ['date', 'org', 'tld', 'ccs', 'bcced', 'mail_type', 'images', 'urls', 'salutations', 'designation', 'chars_in_subject', 'chars_in_body']
-------------
categorical features: ['date', 'org', 'tld', 'mail_type']
-------------
numericalfeatures: ['ccs', 'bcced', 'images', 'urls', 'salutations', 'designation', 'chars_in_subject', 'chars_in_body']


In [16]:
for col in categorical_features:
    print(col,'\n','****************','\n',X[col].value_counts())
    print('*****************************')

date 
 **************** 
 Sat, 04 May 2019 18:17:01 -0700          13
Tue, 22 Sep 2020 03:28:22 -0700          12
Tue, 04 Jun 2019 05:05:20 -0700          12
Sun, 03 Nov 2019 08:51:24 -0800          12
Tue, 12 May 2020 05:53:30 -0700          11
Sun, 09 Jun 2019 04:59:21 -0700          11
Sat, 29 Feb 2020 12:32:34 -0800          11
Tue, 17 Sep 2019 02:44:03 -0700          11
Sat, 06 Jun 2020 08:09:28 -0700          11
Tue, 01 Oct 2019 08:26:18 -0700          11
Sat, 07 Mar 2020 07:41:19 -0800          10
Sat, 27 Jul 2019 01:42:45 -0700          10
Tue, 27 Oct 2020 03:43:04 -0700          10
Tue, 21 Jul 2020 03:03:57 -0700          10
Tue, 10 Mar 2020 08:17:15 -0700          10
Sat, 11 Jul 2020 00:32:14 -0700          10
Tue, 29 Jan 2019 03:49:04 -0800          10
Mon, 01 Jul 2019 06:50:42 -0700          10
Tue, 28 Jan 2020 05:59:46 -0800           9
Tue, 25 Feb 2020 14:34:36 -0800           9
Thu, 12 Dec 2019 15:49:23 -0800           9
Sat, 13 Jun 2020 05:29:06 -0700           9
Tue, 0

In [17]:
for col in columns:
    print(col,'----------------',X[col].isnull().sum())

date ---------------- 0
org ---------------- 1677
tld ---------------- 1679
ccs ---------------- 0
bcced ---------------- 0
mail_type ---------------- 98
images ---------------- 0
urls ---------------- 0
salutations ---------------- 0
designation ---------------- 0
chars_in_subject ---------------- 11
chars_in_body ---------------- 0


In [18]:
from sklearn.impute import SimpleImputer
from sklearn.compose import make_column_transformer, ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder, RobustScaler

In [19]:
## PREPROCESSING

# Put all the strings in lower case
def standardisation(X):
    for col in categorical_features:
        X[col]=X[col].str.lower()

# Changing mail_type columns in more columns
def new_variables(X):
    m=X.shape[0]
    L=X['mail_type'].unique().tolist()
    M=[]
    for l in L:
        M+=(str(l).split('/'))
    keys=set(M)
    #print(keys)
    newColumns={}
    for key in keys:
        temp=[]
        for i in range(m):
            if key in list( str(X['mail_type'].iloc[i]).split('/')):
                temp.append(1)
            else:
                temp.append(0)
        newColumns[key]= temp
    for key in newColumns.keys():
        X[key]=newColumns[key]
        
# Dropping useless features
def dropping(X, cols=['date','mail_type']):
    X.drop(cols, axis=1,inplace=True)

In [20]:
#Strings in lower case
standardisation(X)

print(X['mail_type'].value_counts())

multipart/alternative    28196
text/html                 5595
multipart/mixed           3405
text/plain                1397
multipart/related          902
text/html                   40
multipart/signed            23
multipart/report            12
text/calendar                2
multipart/idm                1
Name: mail_type, dtype: int64


In [21]:
# Changing  mail_type columns in more columns
new_variables(X)

In [22]:
X.columns

Index(['date', 'org', 'tld', 'ccs', 'bcced', 'mail_type', 'images', 'urls',
       'salutations', 'designation', 'chars_in_subject', 'chars_in_body',
       'text', 'alternative', 'multipart', 'html ', 'report', 'calendar',
       'html', 'nan', 'signed', 'plain', 'mixed', 'idm', 'related'],
      dtype='object')

In [23]:
# Dropping useless features
dropping(X)

In [24]:
X.head()

,org,tld,ccs,bcced,images,urls,salutations,designation,chars_in_subject,chars_in_body,...,html,report,calendar,html,nan,signed,plain,mixed,idm,related
0,researchgatemail,net,0,0,4,28,0,1,47.0,25556,...,0,0,0,0,0,0,0,0,0,0
1,no-ip,com,0,0,6,32,0,0,46.0,19930,...,0,0,0,0,0,0,0,0,0,0
2,mail,goodreads.com,0,0,0,0,0,0,21.0,4,...,0,0,0,0,0,0,0,1,0,0
3,insideapple,apple.com,0,0,108,171,0,0,52.0,96568,...,0,0,0,0,0,0,0,0,0,0
4,twitter,com,0,0,20,166,0,0,81.0,95131,...,0,0,0,0,0,0,0,0,0,0


In [25]:
# Dropping more features
dropping(X, cols=['org','tld'])

In [26]:
X.head()

,ccs,bcced,images,urls,salutations,designation,chars_in_subject,chars_in_body,text,alternative,...,html,report,calendar,html,nan,signed,plain,mixed,idm,related
0,0,0,4,28,0,1,47.0,25556,0,1,...,0,0,0,0,0,0,0,0,0,0
1,0,0,6,32,0,0,46.0,19930,0,1,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,21.0,4,0,0,...,0,0,0,0,0,0,0,1,0,0
3,0,0,108,171,0,0,52.0,96568,0,1,...,0,0,0,0,0,0,0,0,0,0
4,0,0,20,166,0,0,81.0,95131,0,1,...,0,0,0,0,0,0,0,0,0,0


In [27]:
def preprocessing(X):
    standardisation(X)
    new_variables(X)
    dropping(X)

In [60]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import SVC

In [61]:
X, y = split_X_y(train_df)
#X, y = X[0:500], y[0:500]
preprocessing(X)
dropping(X, cols=['org','tld'])

In [62]:
# Obtaining the new features
categorical_features=X.select_dtypes(include=['object','category']).columns.tolist()
numerical_features=X.select_dtypes(exclude=['object','category']).columns.tolist()

# Creating the pipeline of the prepocessing
numerical_pipeline=Pipeline(steps=[('imputation',SimpleImputer(strategy='median')),
                                     ('standard',RobustScaler())])

categorical_pipeline=Pipeline(steps=[('imputation',SimpleImputer(strategy='most_frequent')),
                                     ('oneHot',OneHotEncoder())])

preparation=ColumnTransformer(transformers=[('categorical',categorical_pipeline, categorical_features),
                                           ('numerical',numerical_pipeline,numerical_features)])

model = Pipeline(steps=[('preparation', preparation),
                      ('logit', OneVsRestClassifier(SVC(kernel='linear', probability=True)) ) ])

model = Pipeline(steps=[('preparation', preparation),
                      ('logit', OneVsRestClassifier(LogisticRegression()) ) ])

In [63]:
# Splitting train set in train and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 123)

In [64]:
model.fit(X_train, y_train);

C:\Users\matte\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\matte\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\matte\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\matte\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\matte\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence t

In [65]:
# Accuracy of the model
print("Accuracy du modèle sur training set: %.2f" % model.score(X_train, y_train))
print("Accuracy du modèle sur test set: %.2f" % model.score(X_test, y_test))

Accuracy du modèle sur training set: 0.29
Accuracy du modèle sur test set: 0.29


In [66]:
# Mean log loss of the model
log_loss_train = log_loss(y_train, model.predict_proba(X_train))
print("Log-loss du modèle sur training set: %.2f" % log_loss_train)

log_loss_test = log_loss(y_test, model.predict_proba(X_test))
print("Log-loss du modèle sur test set: %.2f" % log_loss_test)

Log-loss du modèle sur training set: 1.96
Log-loss du modèle sur test set: 1.96
